In [1]:
# 0) 설치 (Colab)
!pip -q install transformers==4.42.3 torch --upgrade



In [2]:
# =========================================================
# 1) 모델/토크나이저 로딩 (KoGPT2는 fast 권장)
# =========================================================
import torch, re, random, textwrap
from typing import List, Tuple
from transformers import AutoTokenizer, AutoModelForCausalLM, pipeline

MODEL_NAME = "skt/kogpt2-base-v2"

tokenizer = AutoTokenizer.from_pretrained(MODEL_NAME, use_fast=True)
model     = AutoModelForCausalLM.from_pretrained(MODEL_NAME)

# GPT2 계열은 pad 토큰이 없으므로 eos로 대체
if tokenizer.pad_token is None:
    tokenizer.pad_token = tokenizer.eos_token

DEVICE = 0 if torch.cuda.is_available() else -1

generator = pipeline(
    "text-generation",
    model=model,
    tokenizer=tokenizer,
    device=DEVICE
)


/usr/local/lib/python3.12/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


config.json: 0.00B [00:00, ?B/s]

tokenizer.json: 0.00B [00:00, ?B/s]

pytorch_model.bin:   0%|          | 0.00/513M [00:00<?, ?B/s]

In [3]:
# =========================================================
# 2) 프롬프트 & 유틸
# =========================================================
def build_prompt(keywords: List[str]) -> str:
    """
    키워드 모두 포함, 짧은 소설(5~8문장), 자연스러운 한국어, 과도한 반복 금지 등 지시
    """
    kw_str = ", ".join(keywords)
    guide = (
        "아래 지침을 따르세요.\n"
        "1) 주어진 모든 키워드를 반드시 포함합니다.\n"
        "2) 짧은 소설 형태로 5~8문장으로 씁니다.\n"
        "3) 한국어로 자연스럽게, 과한 수식/반복을 피합니다.\n"
        "4) 기승전결이 느껴지도록 간단한 사건 전개와 여운을 남기세요.\n"
        "5) 마침표로 문장을 또박또박 끝맺습니다.\n\n"
        f"키워드: {kw_str}\n"
        "소설:"
    )
    return guide

def first_n_sentences(text: str, n_min=5, n_max=8) -> str:
    """
    생성 텍스트에서 5~8문장 범위로 잘라 반환 (문장 경계 추정)
    """
    # 문장 경계(간단 추정)
    sents = re.split(r'(?<=[\.!?])\s+', text.strip())
    # 프롬프트 꼬리 제거(혹시 포함된 경우)
    if "소설:" in sents[0]:
        sents[0] = sents[0].split("소설:")[-1].strip()
    sents = [s.strip() for s in sents if s.strip()]
    if not sents:
        return text.strip()

    # 최소 n_min문장 이상 확보, 최대 n_max로 컷
    if len(sents) < n_min:
        clipped = " ".join(sents)
    else:
        clipped = " ".join(sents[:min(len(sents), n_max)])

    # 공백 정리
    clipped = re.sub(r"\s+", " ", clipped).strip()
    return clipped

def contains_all_keywords(text: str, keywords: List[str]) -> bool:
    low = text.lower()
    return all(kw.lower() in low for kw in keywords)


In [4]:
# =========================================================
# 3) 소설 생성 함수 (재시도 루프 포함)
# =========================================================
def generate_short_story(
    keywords: List[str],
    max_new_tokens: int = 220,     # 대략 5~8문장 분량
    temperature: float = 0.9,
    top_p: float = 0.92,
    repetition_penalty: float = 1.12,
    no_repeat_ngram_size: int = 3,
    num_return_sequences: int = 5,
    max_tries: int = 4,
    seed: int = 42
) -> Tuple[str, dict]:
    """
    복수 키워드를 모두 포함하는 짧은 소설 생성.
    실패 시 샘플링 탐색성을 높이며 재시도.
    반환: (소설 텍스트, 메타정보)
    """
    assert len(keywords) > 0, "키워드를 한 개 이상 입력하세요."
    prompt = build_prompt(keywords)

    torch.manual_seed(seed)
    random.seed(seed)

    t, p = temperature, top_p
    last_candidates = None

    for attempt in range(1, max_tries+1):
        outs = generator(
            prompt,
            max_new_tokens=max_new_tokens,
            do_sample=True,
            temperature=t,
            top_p=p,
            repetition_penalty=repetition_penalty,
            no_repeat_ngram_size=no_repeat_ngram_size,
            pad_token_id=tokenizer.eos_token_id,
            eos_token_id=tokenizer.eos_token_id,
            num_return_sequences=num_return_sequences
        )
        last_candidates = outs

        # 후보들 중 키워드 모두 포함한 텍스트를 선택
        for o in outs:
            gen = o["generated_text"]
            story = gen[len(prompt):].strip()
            story = first_n_sentences(story, n_min=5, n_max=8)
            if contains_all_keywords(story, keywords):
                meta = {"attempt": attempt, "temperature": t, "top_p": p}
                return story, meta

        # 재시도 시 탐색성 ↑
        t = min(1.25, t + 0.1)
        p = min(0.98, p + 0.03)

    # 모든 시도에서 키워드 전부 포함 실패 → 가장 자연스러운 후보를 반환(경고 포함)
    fallback = last_candidates[0]["generated_text"][len(prompt):].strip()
    fallback = first_n_sentences(fallback, n_min=5, n_max=8)
    warn = "[참고] 일부 키워드가 빠졌을 수 있습니다.\n" + fallback
    meta = {"attempt": attempt, "temperature": t, "top_p": p}
    return warn, meta


In [5]:
# =========================================================
# 4) 사용 예시
# =========================================================
examples = [
    ["비", "소녀", "들판", "우산"],
    ["바다", "라디오", "여름밤"],
    ["연구실", "노트", "낡은 시계", "비밀"]
]

for kws in examples:
    story, meta = generate_short_story(kws, max_new_tokens=240, num_return_sequences=6, max_tries=4, seed=42)
    print(f"\n=== 키워드: {kws} ===")
    print(textwrap.fill(story, width=90))
    print("메타:", meta)



=== 키워드: ['비', '소녀', '들판', '우산'] ===
[참고] 일부 키워드가 빠졌을 수 있습니다. 산보, 나무, 사람, 추리소설의 특성상 특정 주제를 다루지 않습니다. 따라서 문법을 충실히 하지 않아도 되도록 미리
문단 내에 가둡니다. 가령 문학평론가 이윤택 씨가 쓰고 있다는 신문 칼럼이나, 어떤 내용을 이야기하는 것은 읽지 않고 읽는 것도 추천됩니다. 하지만 독서의
첫인상, 문법, 그 다음이 이야기의 목적. 물론 이 단어나 문장들을 잘 활용하기는 어렵기는 하지만 글의 제목이나 제목을 직접 기억하지 않아도 될 때 읽어도 좋은
책이 없을까요? 우선 이런 식의 책을 많이 읽어두어 보세요. 어떤 것이 중요하기 때문입니다. 일반적으로 글의 주제는 글 전체의 흐름을 잡는데, 이를 문장으로
표현하는 경향이 강한 게 사실입니다만, 여기서는 그렇지 않은 경우도 있을 겁니다 예를 들어, 어떤 내용의 글을 본 사람은 먼저 그것이 '아, 정말...'이라고
말하겠지요.
메타: {'attempt': 4, 'temperature': 1.25, 'top_p': 0.98}

=== 키워드: ['바다', '라디오', '여름밤'] ===
[참고] 일부 키워드가 빠졌을 수 있습니다. 날씨에 민감해지는 현대인이 많아짐에 따라 많은 사람들이 인터넷에서 찾아보는 것들이 있다. 물론 그 중 일부는
일부이지만 최근 유행하고 있는 루틴트들과 같이, 새로운 주제를 소개하는 경우가 많다는 점도 알고 있다. 하지만 이러한 것들은 단순히 글을 쓸 때의 재미 정도에
머물러 있어 문제가 있는 것이라 생각하기 쉬우므로, 그 주의 사항을 주의 깊게 살펴보아야 한다. 특히 최근에는 주로 소설책을 읽는 사람이 많으며, 이를 통해서
글의 내용과 맥락을 직접 확인할 수 있다 (Force of threshood weapon how to rule of a field on the continue?
parts with public opinion). 따라서 이 문항에서는 각 문장의 특징을 분석하고 각각의 문법 규칙을 숙

You seem to be using the pipelines sequentially on GPU. In order to maximize efficiency please use a dataset



=== 키워드: ['연구실', '노트', '낡은 시계', '비밀'] ===
[참고] 일부 키워드가 빠졌을 수 있습니다. 연구실 및 노트 기분 좋고 재미 많은 글 읽는 날(원데이), 즐거운 책, 재미있는 글이 될 듯싶은 글 한 편만 골라
보세요. 예) 여러 가지 신문 칼럼 중에서 어떤 내용을 이야기하겠어? 예) '한국인의 삶에 있어 가장 크게 기여하고 있는 것, 바로 이것'의 글은 무엇을 담고
있을까? 예) '시간의 굴곡과 고통에도 불구하고 최선을 다하는 사람들의 모습'이란 제목으로, 최근 한국 사회에 시사하는 메시지를 소개합니다. 시각의 문제를 다루는
글 가운데 어떤 부분을 중심으로 서술할까? 예) "지난 몇십 년간 우리 사회가 가장 심각한 사회병리현상을 보인 것은, 인간사회의 불균형이 빚은 극심한 불균형
문제"라는 점을 논지로 제시한 논문입니다. 주제 : 한국 사회의 발전 과정에서 나타나는 사회 병리 현상에 대한 분석의 장을 마련하는 데 있어서 주로 한국 사회를
구성하는 구조 속에서 일어나는 갈등 구조와 사회 구조적 갈등의 원인과 원인들을 밝히는 것입니다. 다음 글을 보고 나서, 독자들이 더 나은 미래사회를 향해
나아가기를 바라는 마음을 갖도록 해 드립니다.
메타: {'attempt': 4, 'temperature': 1.25, 'top_p': 0.98}


In [8]:
# =========================================================
# 5) 간단 입력 인터페이스
# =========================================================
user = "학교, 방학, 숙제, 학생"  # 콤마로 구분해 입력
keywords = [w.strip() for w in user.split(",") if w.strip()]
story, meta = generate_short_story(keywords, max_new_tokens=240, num_return_sequences=6, max_tries=4, seed=123)
print("\n[사용자 입력 결과]")
print(textwrap.fill(story, width=90))
print("메타:", meta)



[사용자 입력 결과]
[참고] 일부 키워드가 빠졌을 수 있습니다. 취업, 졸업 등등 등 작품: 학과/공부, 사회, 경제, 문화, 체육, 게임 및 놀이 등 주요 문단에서는 우선 중요한
키워드만 모아 둡니다. 가볍게 넘어갈 필요는 없다. 당신이 원하는 바를 정확하게 적는 것이다. 어른스럽게 글을 쓰면 당신은 많은 지식을 요구할 것이 분명하다.
그래서 한 문장보다 여러 문장을 몇 글자씩 적을 때는 불필요한 단어를 지우는 작업을 자주 하게 된다. 그러나 이런 과정보다는 문맥적으로 중요한 요소만을 골라 쓰는
연습은 더욱 효과적이다. 처음엔 많이 쓰라고 말하다가 조금만 더 잘 썼더라면 내용을 많이 알 수 있었을 텐데 그렇지 않다면 너무 큰 소리로 말했을 뿐이므로 주의.
특히 특정 주제나 인물의 성격 묘사가 들어가기에 더 그러하다.
메타: {'attempt': 4, 'temperature': 1.25, 'top_p': 0.98}
